# 📈 6강: 성적 분석 & 위험 감지 (Risk Radar)

> matplotlib 차트 + AI 분석으로 위험 학생을 자동 감지합니다.

## ⚠️ 시작 전 필수! 노트북 복사하기

이 노트북은 **읽기 전용**입니다. 실습하려면 반드시 복사본을 만드세요!

```
📌 상단 메뉴 → 파일 → 드라이브에 사본 저장
```

복사본이 내 Google Drive에 저장되면, 그 파일에서 코드를 수정하고 실행할 수 있습니다.

In [ ]:
# ⚙️ 환경 설정
!pip install -q -U google-genai

from google import genai
from google.colab import userdata
import getpass
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'DejaVu Sans'

try:
    api_key = userdata.get('GEMINI_API_KEY')
    if isinstance(api_key, dict):
        api_key = str(list(api_key.values())[0])
    API_KEY = str(api_key).strip()
except Exception:
    API_KEY = getpass.getpass('🔑 Gemini API Key를 입력하세요: ')

client = genai.Client(api_key=API_KEY)
MODEL = 'gemini-2.5-flash'
print('✅ 준비 완료!')

In [ ]:
# 📊 6개월 성적 데이터
data = {
    'name': ['Kim SY', 'Park JH', 'Lee HE', 'Choi MJ', 'Jung SA'],
    'month1': [85, 70, 92, 78, 88],
    'month2': [82, 68, 90, 75, 85],
    'month3': [78, 72, 88, 70, 82],
    'month4': [75, 65, 91, 68, 80],
    'month5': [72, 60, 93, 65, 78],
    'month6': [70, 58, 95, 62, 75]
}
df = pd.DataFrame(data)
print('📋 학생 성적 데이터:')
print(df.to_string(index=False))

## 🔬 실습 1: 성적 추이 차트

In [ ]:
months = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']
score_cols = ['month1', 'month2', 'month3', 'month4', 'month5', 'month6']

fig, ax = plt.subplots(figsize=(10, 6))

for _, row in df.iterrows():
    scores = [row[c] for c in score_cols]
    trend = scores[-1] - scores[0]
    color = '#e74c3c' if trend < -5 else '#2ecc71' if trend > 5 else '#95a5a6'
    marker = 'v' if trend < -5 else '^' if trend > 5 else 'o'
    ax.plot(months, scores, marker=marker, linewidth=2, color=color,
            label=f"{row['name']} ({trend:+d})")

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Student Score Trends (6 Months)', fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_ylim(50, 100)
ax.grid(True, alpha=0.3)
ax.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Warning Line')

plt.tight_layout()
plt.savefig('score_trends.png', dpi=150, bbox_inches='tight')
plt.show()
print('📊 차트 저장 완료!')

## 🔬 실습 2: 위험 학생 자동 감지

In [ ]:
print('🚨 위험 학생 분석')
print('='*50)

at_risk = []
for _, row in df.iterrows():
    scores = [row[c] for c in score_cols]
    trend = scores[-1] - scores[0]
    recent = scores[-1]
    drop_3m = scores[-3] - scores[-1]

    risk_level = 'safe'
    reasons = []

    if trend <= -10:
        risk_level = 'high'
        reasons.append(f'6M trend: {trend:+d}')
    elif trend <= -5:
        risk_level = 'medium'
        reasons.append(f'6M trend: {trend:+d}')

    if recent < 65:
        risk_level = 'high'
        reasons.append(f'Recent: {recent}')

    if drop_3m >= 8:
        risk_level = 'high' if risk_level != 'safe' else 'medium'
        reasons.append(f'3M drop: -{drop_3m}')

    status = {'high': '🔴 HIGH', 'medium': '🟡 MED', 'safe': '🟢 OK'}[risk_level]
    print(f"{status} {row['name']:12s} | Latest: {recent} | 6M: {trend:+d} | {', '.join(reasons) if reasons else 'Stable'}")

    if risk_level != 'safe':
        at_risk.append({'name': row['name'], 'level': risk_level, 'score': recent, 'trend': trend})

print(f'\n⚠️ 위험 학생: {len(at_risk)}명')

## 🔬 실습 3: AI 분석 리포트

In [ ]:
risk_summary = '\n'.join([f"- {r['name']}: {r['level']} risk, score {r['score']}, trend {r['trend']:+d}" for r in at_risk])

prompt = f"""
너는 교육 데이터 분석가이자 학습 코치야.

아래 위험 학생 데이터를 분석하고 개입 전략을 세워줘.

위험 학생:
{risk_summary}

각 학생별로:
1. 📊 원인 분석 (추정 3가지)
2. 🎯 즉시 조치 (이번 주 할 일)
3. 📅 4주 회복 계획
4. 💬 학부모 메시지 (카톡 형식, 100자)

데이터 기반으로 구체적으로.
"""

response = client.models.generate_content(model=MODEL, contents=prompt)
ai_report = response.text
print('🤖 AI 분석 리포트')
print('='*50)
print(ai_report)

In [ ]:
# 💾 저장
with open('위험학생_리포트.txt', 'w', encoding='utf-8') as f:
    f.write(ai_report)
from google.colab import files
files.download('위험학생_리포트.txt')
files.download('score_trends.png')
print('✅ 다운로드!')

## 🏆 6강 완료!
👉 [7강: 퇴원생 관리 & 재등록](https://colab.research.google.com/github/Reasonofmoon/cedu-1/blob/main/notebooks/07_re_enroll.ipynb)